In [2]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import numpy as np
from sklearn.metrics import *
import pandas as pd

In [3]:
# %%capture
fclip = FashionCLIP('fashion-clip')

In [4]:
annotations = pd.read_csv('D:\M1/fashion/lda_model/data/anotation_new.csv')
img_path = annotations.iloc[:, 0]
images = img_path.values

In [5]:
item_set = set()
new_images = []
for i in images:
    item_id = i.split('/')[-1]
    if item_id in item_set:
        continue
    new_images.append(i)
    item_set.add(item_id)
images = new_images

In [6]:
# images = images[:len(images)// 100]
images = [i.replace('\\', '/') for i in images]

In [9]:
image_embeddings = fclip.encode_images(images, batch_size=128)

  0%|          | 37/16159 [02:43<19:50:18,  4.43s/it]
4040it [1:50:03,  1.63s/it]                            


In [10]:
image_embeddings = image_embeddings/np.linalg.norm(image_embeddings, ord=2, axis=-1, keepdims=True)

In [16]:
category = ['Vest top', 'Hair/alice_band', 'Leggings/Tights', 'T-shirt', 'Sneakers',
 'Sunglasses', 'Cardigan', 'Gloves', 'Underwear_Tights', 'Hoodie', 'Other_shoe',
 'Shorts' ,'Jumpsuit/Playsuit', 'Dress', 'Trousers', 'Belt', 'Socks',
 'Underwear_bottom' ,'Bodysuit' ,'Hat/beanie', 'Scarf' ,'Jacket',
 'Other_accessories', 'Bra' ,'Swimwear_bottom' ,'Blazer' ,'Top' ,'Polo shirt',
 'Sweater' ,'Necklace' ,'Pyjama_set' ,'Blouse', 'Bag', 'Shirt' ,'Coat' ,'Boots',
 'Skirt', 'Garment_Set', 'Bikini_top', 'Sandals' ,'Dungarees', 'Earring',
 'Cap/peaked' ,'Ballerinas', 'Swimsuit' ,'Hat/brim']
materials = ['Cotton', 'Polyester', 'Rayon', 'Linen', 'Wool', 'Silk', 'Nylon', 'Polyurethane', 'Denim', 'Spandex']
colors = ['Black', 'White', 'Gray', 'Navy', 'Blue', 'Red', 'Green', 'Brown', 'Beige', 'Pink',
          'Yellow', 'Orange', 'Purple', 'Lavender', 'Teal', 'Turquoise', 'Magenta', 'Olive', 'Maroon', 'Charcoal']

In [17]:
created_material_labels = []
created_color_labels = []
for c in category:
    for color in colors:
        created_color_labels.append(f'{c}_{color}')
    for m in materials:
        created_material_labels.append(f'{c}_{m}')
created_color_labels

['Vest top_Black',
 'Vest top_White',
 'Vest top_Gray',
 'Vest top_Navy',
 'Vest top_Blue',
 'Vest top_Red',
 'Vest top_Green',
 'Vest top_Brown',
 'Vest top_Beige',
 'Vest top_Pink',
 'Vest top_Yellow',
 'Vest top_Orange',
 'Vest top_Purple',
 'Vest top_Lavender',
 'Vest top_Teal',
 'Vest top_Turquoise',
 'Vest top_Magenta',
 'Vest top_Olive',
 'Vest top_Maroon',
 'Vest top_Charcoal',
 'Hair/alice band_Black',
 'Hair/alice band_White',
 'Hair/alice band_Gray',
 'Hair/alice band_Navy',
 'Hair/alice band_Blue',
 'Hair/alice band_Red',
 'Hair/alice band_Green',
 'Hair/alice band_Brown',
 'Hair/alice band_Beige',
 'Hair/alice band_Pink',
 'Hair/alice band_Yellow',
 'Hair/alice band_Orange',
 'Hair/alice band_Purple',
 'Hair/alice band_Lavender',
 'Hair/alice band_Teal',
 'Hair/alice band_Turquoise',
 'Hair/alice band_Magenta',
 'Hair/alice band_Olive',
 'Hair/alice band_Maroon',
 'Hair/alice band_Charcoal',
 'Leggings/Tights_Black',
 'Leggings/Tights_White',
 'Leggings/Tights_Gray',
 'Leg

In [18]:
label_material_embeddings = fclip.encode_text(created_material_labels, batch_size=32)
label_material_embeddings = label_material_embeddings/np.linalg.norm(label_material_embeddings, ord=2, axis=-1, keepdims=True)
label_color_embeddings = fclip.encode_text(created_color_labels, batch_size=32)
label_color_embeddings = label_color_embeddings/np.linalg.norm(label_color_embeddings, ord=2, axis=-1, keepdims=True)

15it [00:01, 12.23it/s]                        
29it [00:02, 12.89it/s]                        


In [19]:
# predicteds
predicted_materials = label_material_embeddings.dot(image_embeddings.T)
predicted_materials = [created_material_labels[k] for k in np.argmax(predicted_materials, axis=0)]
predicted_colors = label_color_embeddings.dot(image_embeddings.T)
predicted_colors = [created_color_labels[k] for k in np.argmax(predicted_colors, axis=0)]

In [20]:
with open('D:\M1/fashion/lda_model/data/fashion_clip.txt', 'w') as f:
    for img_path, m, c in zip(images, predicted_materials, predicted_colors):
        # item_id = i.split('/')[-1]
        item_id = img_path.split('/')[-1][:-6]
        
        f.write(f'{item_id} {m} {c} \n')

In [22]:
predicted_colors[:10]
created_color_labels

['Vest top_Black',
 'Vest top_White',
 'Vest top_Gray',
 'Vest top_Navy',
 'Vest top_Blue',
 'Vest top_Red',
 'Vest top_Green',
 'Vest top_Brown',
 'Vest top_Beige',
 'Vest top_Pink',
 'Vest top_Yellow',
 'Vest top_Orange',
 'Vest top_Purple',
 'Vest top_Lavender',
 'Vest top_Teal',
 'Vest top_Turquoise',
 'Vest top_Magenta',
 'Vest top_Olive',
 'Vest top_Maroon',
 'Vest top_Charcoal',
 'Hair/alice band_Black',
 'Hair/alice band_White',
 'Hair/alice band_Gray',
 'Hair/alice band_Navy',
 'Hair/alice band_Blue',
 'Hair/alice band_Red',
 'Hair/alice band_Green',
 'Hair/alice band_Brown',
 'Hair/alice band_Beige',
 'Hair/alice band_Pink',
 'Hair/alice band_Yellow',
 'Hair/alice band_Orange',
 'Hair/alice band_Purple',
 'Hair/alice band_Lavender',
 'Hair/alice band_Teal',
 'Hair/alice band_Turquoise',
 'Hair/alice band_Magenta',
 'Hair/alice band_Olive',
 'Hair/alice band_Maroon',
 'Hair/alice band_Charcoal',
 'Leggings/Tights_Black',
 'Leggings/Tights_White',
 'Leggings/Tights_Gray',
 'Leg